In [33]:
%use dataframe, kandy

# How are we doing?
We wouldn't be working agile if we didn't have some ways to gather timely feedback that drive us towards improvements.

For this we'll use "Aging Charts".

In [34]:
import kotlinx.datetime.daysUntil
import org.jetbrains.kotlinx.dataframe.api.dropNulls

val csv = DataFrame.read("data/a_team.csv")

In [35]:
val cleaned = csv.dropNulls { `In Analysis` and Analyzed and `In Development` and Developed and `In Acceptance` and `In Production` }
cleaned

StoryId,Selected,Refinement Started,Backlogged,In Analysis,Analyzed,In Development,Developed,In Acceptance,In Production,Abandoned
ST-0001,2023-01-01T17:00,2023-01-04T10:09,2023-01-09T17:46,2023-01-10T16:03,2023-01-15T18:00,2023-01-16T09:56,2023-01-26T08:15,2023-01-29T12:02,2023-02-07T16:28,null
ST-0002,2023-01-08T17:32,2023-01-11T12:28,2023-01-16T16:51,2023-01-16T17:21,2023-01-16T17:51,2023-01-18T10:26,2023-01-27T14:28,2023-01-29T09:42,2023-02-11T14:04,null
ST-0003,2023-01-13T10:29,2023-01-14T09:25,2023-01-16T10:42,2023-01-19T10:45,2023-01-20T16:25,2023-01-22T13:05,2023-01-29T08:21,2023-02-01T15:28,2023-02-04T14:21,null
ST-0004,2023-01-09T09:16,2023-01-10T08:57,2023-01-14T12:48,2023-01-18T14:54,2023-01-22T14:09,2023-01-24T13:05,2023-02-02T08:51,2023-02-06T14:57,2023-02-16T12:01,null
ST-0005,2023-01-16T09:29,2023-01-16T13:35,2023-01-22T12:39,2023-01-22T13:09,2023-01-24T09:10,2023-01-25T12:40,2023-02-01T16:48,2023-02-01T17:18,2023-02-04T18:00,null
ST-0006,2023-01-18T08:46,2023-01-19T16:12,2023-01-20T15:15,2023-01-24T09:42,2023-01-26T14:42,2023-01-28T16:19,2023-02-05T11:21,2023-02-09T18:00,2023-02-23T14:22,null
ST-0007,2023-01-23T08:05,2023-01-26T16:42,2023-01-26T17:15,2023-01-28T08:29,2023-01-28T10:17,2023-01-30T13:35,2023-02-08T11:02,2023-02-08T11:22,2023-02-13T08:21,null
ST-0008,2023-01-26T08:05,2023-01-26T09:09,2023-01-29T17:02,2023-01-30T08:19,2023-02-02T18:00,2023-02-04T10:42,2023-02-09T14:48,2023-02-09T15:18,2023-02-18T17:41,null
ST-0009,2023-01-28T17:53,2023-01-31T18:00,2023-02-02T18:00,2023-02-02T18:30,2023-02-06T08:08,2023-02-07T15:35,2023-02-19T18:00,2023-02-21T18:00,2023-02-24T18:00,null
ST-0010,2023-01-29T09:38,2023-02-03T13:16,2023-02-09T12:39,2023-02-12T10:00,2023-02-14T08:31,2023-02-18T18:00,2023-03-02T11:43,2023-03-02T12:45,2023-04-13T12:29,null


# Current state of our KanBan board
We have 3 Work items In Progress:

ST-0179 has been in progress for 36 days and is currently In Acceptance.

ST-0180 has been in progress for 4 days and is currently In Development.

ST-0181 has been in progress for 4 days and is currently In Analysis.


# Can we improve anything?

How can we know?

In [36]:
val offsetWhenFinishedSameDay = 1

val valueStream = cleaned
    .add("AnalysisTime") { `In Analysis`.date.daysUntil(Analyzed.date) + offsetWhenFinishedSameDay }
    .add("CycleTime") { `In Analysis`.date.daysUntil(`In Production`.date) + offsetWhenFinishedSameDay }
    .select("StoryId", "AnalysisTime","CycleTime")
valueStream

StoryId,AnalysisTime,CycleTime
ST-0001,6,29
ST-0002,1,27
ST-0003,2,17
ST-0004,5,30
ST-0005,3,14
ST-0006,3,31
ST-0007,1,17
ST-0008,4,20
ST-0009,5,23
ST-0010,3,61


In [37]:
val percentiles = valueStream.aggregate {
    percentile(50.0) { AnalysisTime } into "p50Analysis"
    percentile(85.0) { AnalysisTime } into "p85Analysis"
    percentile(50.0) { CycleTime } into "p50CycleTime"
    percentile(85.0) { CycleTime } into "p85CycleTime"
}
percentiles

p50Analysis,p85Analysis,p50CycleTime,p85CycleTime
"4,000000","6,000000","24,000000","33,000000"


In [38]:
import org.jetbrains.kotlinx.kandy.ir.Plot
import org.jetbrains.kotlinx.statistics.distribution.NormalDistribution
import org.jetbrains.letsPlot.Stat
import org.jetbrains.letsPlot.core.spec.plotson.BinStatOptions
import org.jetbrains.letsPlot.core.spec.plotson.theme

val analysisAgingChart = plot(valueStream) {
    layout { size = 2000 to 900 }
    points {
        y(AnalysisTime)
        x(StoryId)
    }
    hLine { yIntercept.constant(percentiles["p50Analysis"]); type = LineType.DASHED }
    hLine { yIntercept.constant(percentiles["p85Analysis"]); type = LineType.DASHED }
}
analysisAgingChart

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="AuotsP"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
"StoryId":["ST-0001","ST-0002","ST-0003","ST-0004","ST-0005","ST-0006","ST-0007","ST-0008","ST-0009","ST-0010","ST-0011","ST-0012","ST-0014","ST-0015","ST-0016","ST-0017","ST-0018","ST-0020","ST-0021","ST-0022","ST-0023","ST-0024","ST-0026","ST-0027","ST-0028","ST-0029","ST-0030","ST-0031","ST-0032","ST-0033","ST-0034","ST-0035","ST-0036","ST-0037","ST-0038","ST-0040","ST-0041","ST-0042","ST-0043","ST-0044","ST-0045","ST-0046","ST-0047","ST-0048","ST-0049","ST-0050","ST-0051","ST-0052","ST-0053","ST-0054","ST-0055","ST-0056","ST-0057","ST-0058","ST-0059","ST-0060","ST-0061","ST-0062","ST-0063","ST-0064","ST-0065","ST-0066","ST-0067","ST-0068","ST-0069","ST-0070","ST-0071","ST-0072","ST-0073","ST-0074","ST-0075","ST-0076","ST-0077","ST-0078","ST-0079","ST-0080","ST-0081","ST-0082","ST-0084","ST-0085","ST-0086","ST-0087","ST-0088","ST-0089","ST-0090","ST-0091","ST-0092","ST-0093","ST-0095","ST-0096","ST-0097","ST-0098","ST-0099","ST-0100","ST-0101","ST-0103","ST-0104","ST-0105","ST-0106","ST-0107","ST-0108","ST-0109","ST-0110","ST-0111","ST-0112","ST-0113","ST-0114","ST-0115","ST-0116","ST-0117","ST-0118","ST-0119","ST-0120","ST-0121","ST-0122","ST-0123","ST-0124","ST-0125","ST-0126","ST-0127","ST-0128","ST-0129","ST-0130","ST-0131","ST-0132","ST-0133","ST-0134","ST-0135","ST-0136","ST-0137","ST-0139","ST-0140","ST-0141","ST-0142","ST-0143","ST-0144","ST-0145","ST-0146","ST-0147","ST-0148","ST-0149","ST-0150","ST-0151","ST-0152","ST-0153","ST-0154","ST-0155","ST-0156","ST-0157","ST-0158","ST-0159","ST-0160","ST-0161","ST-0162","ST-0163","ST-0164","ST-0165","ST-0166","ST-0168","ST-0169","ST-0170","ST-0171","ST-0172","ST-0173","ST-0174","ST-0175","ST-0176","ST-0177","ST-0178","ST-0179","ST-0180"],
"AnalysisTime":[6.0,1.0,2.0,5.0,3.0,3.0,1.0,4.0,5.0,3.0,1.0,4.0,5.0,1.0,4.0,3.0,4.0,4.0,4.0,4.0,1.0,1.0,4.0,4.0,4.0,4.0,4.0,3.0,4.0,6.0,7.0,3.0,1.0,4.0,5.0,5.0,1.0,4.0,4.0,1.0,5.0,6.0,3.0,4.0,6.0,5.0,3.0,5.0,5.0,3.0,2.0,5.0,6.0,4.0,4.0,3.0,5.0,3.0,3.0,2.0,2.0,4.0,2.0,3.0,2.0,4.0,5.0,4.0,6.0,4.0,3.0,6.0,6.0,6.0,5.0,6.0,5.0,4.0,1.0,4.0,4.0,2.0,1.0,4.0,3.0,2.0,5.0,6.0,3.0,2.0,4.0,6.0,2.0,2.0,3.0,4.0,6.0,1.0,5.0,4.0,5.0,5.0,7.0,3.0,2.0,5.0,6.0,2.0,2.0,6.0,3.0,4.0,5.0,2.0,6.0,5.0,6.0,5.0,5.0,2.0,5.0,3.0,6.0,4.0,1.0,1.0,1.0,2.0,3.0,4.0,2.0,5.0,6.0,3.0,5.0,5.0,2.0,2.0,4.0,5.0,4.0,3.0,4.0,5.0,2.0,1.0,6.0,2.0,4.0,2.0,6.0,6.0,4.0,2.0,5.0,2.0,7.0,5.0,7.0,4.0,1.0,3.0,2.0,1.0,4.0,2.0,2.0,4.0,6.0,2.0,3.0]
},
"ggsize":{
"width":2000.0,
"height":900.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
}],
"layers":[{
"mapping":{
"y":"AnalysisTime",
"x":"StoryId"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"point",
"data":{
}
},{
"mapping":{
},
"stat":"identity",
"yintercept":4.0,
"linetype":"dashed",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"hline",
"data":{
}
},{
"mapping":{
},
"stat":"identity",
"yintercept":6.0,
"linetype":"dashed",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"hline",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"int",
"column":"AnalysisTime"
},{
"type":"str",
"column":"StoryId"
}]
},
"spec_id":"56"
};
 var containerDiv = document.getElementById("AuotsP");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 2000.0,
 height: 900.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 to

In [39]:
val agingChart = plot(valueStream) {
    layout { size = 2000 to 900 }
    points {
        y(CycleTime)
        x(StoryId)
    }
    // vertical markers
    hLine { yIntercept.constant(percentiles["p50CycleTime"]); type = LineType.DASHED }
    hLine { yIntercept.constant(percentiles["p85CycleTime"]); type = LineType.DASHED }
}
agingChart

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="0HX5QT"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
"CycleTime":[29.0,27.0,17.0,30.0,14.0,31.0,17.0,20.0,23.0,61.0,19.0,34.0,33.0,18.0,61.0,61.0,17.0,61.0,28.0,13.0,18.0,23.0,61.0,61.0,28.0,31.0,22.0,12.0,31.0,29.0,18.0,61.0,18.0,34.0,26.0,25.0,24.0,22.0,25.0,11.0,24.0,35.0,20.0,20.0,21.0,15.0,23.0,31.0,22.0,34.0,15.0,19.0,38.0,14.0,61.0,23.0,21.0,61.0,29.0,11.0,30.0,61.0,28.0,30.0,17.0,22.0,26.0,13.0,22.0,30.0,18.0,28.0,26.0,24.0,29.0,29.0,17.0,20.0,28.0,25.0,26.0,13.0,24.0,23.0,27.0,21.0,33.0,25.0,31.0,29.0,20.0,23.0,25.0,19.0,16.0,28.0,38.0,16.0,22.0,23.0,19.0,27.0,23.0,27.0,20.0,19.0,28.0,61.0,61.0,28.0,22.0,61.0,18.0,18.0,28.0,20.0,24.0,29.0,30.0,28.0,25.0,24.0,24.0,16.0,16.0,21.0,14.0,15.0,16.0,61.0,14.0,22.0,27.0,31.0,31.0,25.0,17.0,23.0,16.0,24.0,16.0,61.0,27.0,29.0,61.0,30.0,20.0,17.0,25.0,61.0,22.0,25.0,24.0,21.0,24.0,11.0,21.0,23.0,17.0,17.0,24.0,60.0,17.0,16.0,23.0,18.0,25.0,30.0,35.0,21.0,24.0],
"StoryId":["ST-0001","ST-0002","ST-0003","ST-0004","ST-0005","ST-0006","ST-0007","ST-0008","ST-0009","ST-0010","ST-0011","ST-0012","ST-0014","ST-0015","ST-0016","ST-0017","ST-0018","ST-0020","ST-0021","ST-0022","ST-0023","ST-0024","ST-0026","ST-0027","ST-0028","ST-0029","ST-0030","ST-0031","ST-0032","ST-0033","ST-0034","ST-0035","ST-0036","ST-0037","ST-0038","ST-0040","ST-0041","ST-0042","ST-0043","ST-0044","ST-0045","ST-0046","ST-0047","ST-0048","ST-0049","ST-0050","ST-0051","ST-0052","ST-0053","ST-0054","ST-0055","ST-0056","ST-0057","ST-0058","ST-0059","ST-0060","ST-0061","ST-0062","ST-0063","ST-0064","ST-0065","ST-0066","ST-0067","ST-0068","ST-0069","ST-0070","ST-0071","ST-0072","ST-0073","ST-0074","ST-0075","ST-0076","ST-0077","ST-0078","ST-0079","ST-0080","ST-0081","ST-0082","ST-0084","ST-0085","ST-0086","ST-0087","ST-0088","ST-0089","ST-0090","ST-0091","ST-0092","ST-0093","ST-0095","ST-0096","ST-0097","ST-0098","ST-0099","ST-0100","ST-0101","ST-0103","ST-0104","ST-0105","ST-0106","ST-0107","ST-0108","ST-0109","ST-0110","ST-0111","ST-0112","ST-0113","ST-0114","ST-0115","ST-0116","ST-0117","ST-0118","ST-0119","ST-0120","ST-0121","ST-0122","ST-0123","ST-0124","ST-0125","ST-0126","ST-0127","ST-0128","ST-0129","ST-0130","ST-0131","ST-0132","ST-0133","ST-0134","ST-0135","ST-0136","ST-0137","ST-0139","ST-0140","ST-0141","ST-0142","ST-0143","ST-0144","ST-0145","ST-0146","ST-0147","ST-0148","ST-0149","ST-0150","ST-0151","ST-0152","ST-0153","ST-0154","ST-0155","ST-0156","ST-0157","ST-0158","ST-0159","ST-0160","ST-0161","ST-0162","ST-0163","ST-0164","ST-0165","ST-0166","ST-0168","ST-0169","ST-0170","ST-0171","ST-0172","ST-0173","ST-0174","ST-0175","ST-0176","ST-0177","ST-0178","ST-0179","ST-0180"]
},
"ggsize":{
"width":2000.0,
"height":900.0
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
}],
"layers":[{
"mapping":{
"y":"CycleTime",
"x":"StoryId"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"point",
"data":{
}
},{
"mapping":{
},
"stat":"identity",
"yintercept":24.0,
"linetype":"dashed",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"hline",
"data":{
}
},{
"mapping":{
},
"stat":"identity",
"yintercept":33.0,
"linetype":"dashed",
"sampling":"none",
"inherit_aes":false,
"position":"identity",
"geom":"hline",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"int",
"column":"CycleTime"
},{
"type":"str",
"column":"StoryId"
}]
},
"spec_id":"59"
};
 var containerDiv = document.getElementById("0HX5QT");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height

# How good busy are we?
Here's our current state again.

## ST-0179 has been in progress for **36 days** and is currently In Acceptance.

According to our Aging Chart our 50th percentile is 24 days "in progress", and the 85th percentile is 33.
We sure don't want this story to mess up our stats!
So let's get it to production asap!
This needs to be our first priority.

## ST-0180 has been in progress for 4 days and is currently In Development.
This story is currently well below the 50th percentile of 24 days, and it's right on the median of our analysis time, so probably no action required today.

## ST-0181 has been in progress for 4 days and is currently In Analysis.
Given that our median is 4 days and 85th is 6 days, we should look into finishing analysis for this one today.
If your gut says we won't be done today, let's put effort into splitting up this story today instead.